In [1]:
import matplotlib.pyplot as pp
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.filter import matched_filter
from pycbc.waveform import get_td_waveform
from pycbc.waveform import get_fd_waveform
import numpy as np
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
import pandas as pd
from pycbc.filter import sigma
import h5py
from pycbc.types import timeseries
from pycbc.types import TimeSeries

In [2]:
def gen_noise(psd_need):
    
    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    if (psd_need == True) :
        pp.plot(psd.sample_frequencies,psd.data)
        pp.loglog()
        pp.xlim(30,2000)
        pp.ylabel('$Strain^2 / Hz$')
        pp.xlabel('Frequency (Hz)')
        pp.show()


    # Generate 256 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(256/ delta_t)
    ts = pycbc.noise.gaussian.noise_from_psd(tsamples,delta_t, psd, seed=127)
    
    
    #plotting noise 
#     pp.plot(ts.sample_times, ts)
#     pp.ylabel('Strain')
#     pp.xlabel('Time (simport numpy as np

#     pp.show()
    

    return ts



In [3]:
def gen_waveform(model,m1,m2,s1z,s2z,distance,time,l):
    
    from pycbc.detector import Detector
    from pycbc.waveform import get_td_waveform


   
    ra = 1.7
    dec = 1.7
    pol = 0.2
    inclination_1 = 0




    # We can calcualate the antenna pattern for Hanford at
    # the specific sky location
    d = Detector("H1")

    # We get back the fp and fc antenna pattern weights.
    fp, fc = d.antenna_pattern(ra, dec, pol, time)
    #print("fp={}, fc={}".format(fp, fc))

    # These factors allow us to project a signal into what the detector would observe
    # IMRPhenomPv2
    # Generate the waveform for the BNS merger
    hp_1, hc_1 = get_td_waveform(approximant=model,
                             mass1=m1, mass2=m2,spin1z=s1z,spin2z=s2z,
                             distance=distance, inclination=inclination_1,
                             delta_t=1.0/4096, f_lower=30)

    ## Apply the factors to get the detector frame strain
    ht_1 = fp * hp_1 + fc * hc_1
    
    
#     pp.plot(ht_1.sample_times, ht_1, label=l)
#     pp.legend()
#     pp.ylabel('Strain')
#     pp.xlabel('Time')
#     pp.grid()
#     pp.show()
    
    
    
    return ht_1


In [4]:
def overlap(ht_1,ht_2,buffer):

    if len(ht_1) < len(ht_2):
        ht_small, ht_big = ht_1, ht_2
    else:
        ht_small, ht_big = ht_2, ht_1

    dt = ht_small.delta_t
   
    
    # make buffer int multiple of dt
    buffer = np.round(buffer*ht_big.sample_rate)/ht_big.sample_rate
    
    
    # elongate the long template to accomodate enough space on left
    ht_big.resize(len(ht_big)+int(buffer*ht_big.sample_rate))
#     pp.plot(ht_big.sample_times,ht_big)
#     pp.show()
    
    # match small template
    ht_small.resize(len(ht_big))
#     pp.plot(ht_small.sample_times,ht_small)
#     pp.show()
    
    
    # correct the position after resizing
    ht_big = ht_big.cyclic_time_shift(buffer)
#     pp.plot(ht_big.sample_times,ht_big)
#     pp.show()
    
    #Shifting the merger time
    ht_small = ht_small.cyclic_time_shift(ht_small.start_time-ht_big.start_time-buffer)
#     pp.plot(ht_small.sample_times,ht_small)
#     pp.show()
    
    #Equating the start time of both signals
    ht_small.start_time = ht_big.start_time
   
    
    # resample to original sample rate
    ht_big = ht_big.resample(dt)
    ht_small = ht_small.resample(dt)
    
    

    #Combining the signals
    ht_total = ht_small + ht_big
#     pp.plot(ht_total.sample_times,ht_total)
#     pp.show()
    return ht_total,ht_2

In [5]:
def inject(ht_total,ts):
    
    #Resizing the signal
    ht_total.resize(len(ts)) 
    
    #Equating the start time
    ht_total.start_time = ts.start_time
    
    #Injecting signal into noise
    ts = ts.add_into(ht_total) 
    
    #plotting the result
#     pp.plot(ts.sample_times, ts)
#     pp.xlabel('Time(s)')
#     pp.ylabel('Strain')
#     pp.grid()
#     pp.show()
    
    return ts 



In [6]:
def psd_data(conditioned):

    from pycbc.psd import interpolate, inverse_spectrum_truncation
    # Estimate the power spectral density

    # We use 4 second samples of our time series in Welch method.
    psd = conditioned.psd(4)

    # Now that we have the psd we need to interpolate it to match our data
    # and then limit the filter length of 1 / PSD. After this, we can
    # directly use this PSD to filter the data in a controlled manner

    psd = interpolate(psd, conditioned.delta_f)

    # 1/PSD will now act as a filter with an effective length of 4 seconds
    # Since the data has been highpassed above 30 Hz, and will have low values
    # below this we need to informat the function to not include frequencies
    # below this frequency. 
    psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),
                                      low_frequency_cutoff=30)

#     pp.loglog(psd.sample_frequencies, psd)
#     pp.ylabel('$Strain^2 / Hz$')
#     pp.xlabel('Frequency (Hz)')
#     pp.xlim(30, 1024)
    
    return psd

In [1]:
def gen_template(model,m1,m2,s1z,s2z,conditioned):
    
    from pycbc.detector import Detector
    from pycbc.waveform import get_td_waveform

    
    ra = 1.7
    dec = 1.7
    pol = 0.2
    inclination_1 = 0
    time=0.0



    # We can calcualate the antenna pattern for Hanford at
    # the specific sky location
    d = Detector("H1")

    # We get back the fp and fc antenna pattern weights.
    fp, fc = d.antenna_pattern(ra, dec, pol, time)
    
    
    
    
    hp, hc = get_td_waveform(approximant=model,
                     mass1=m1,
                     mass2=m2,spin1z=s1z,spin2z=s2z,
                     delta_t=conditioned.delta_t,
                     f_lower=30)
    
    
    ht_template = fp * hp + fc * hc
    
    #Resizing the template to match data
    ht_template.resize(len(conditioned))
    
    #Time shift 
    template = ht_template.cyclic_time_shift(ht_template.start_time)
    
   
    return template


In [8]:
def gen_SNR(template,conditioned,psd):
    from pycbc.filter import matched_filter
    import numpy

    snr = matched_filter(template, conditioned,
                         psd=psd, low_frequency_cutoff=30)



    # The `matched_filter` function actually returns a 'complex' SNR.
    # What that means is that the real portion correponds to the SNR
    # associated with directly filtering the template with the data.
    # The imaginary portion corresponds to filtering with a template that
    # is 90 degrees out of phase. Since the phase of a signal may be 
    # anything, we choose to maximize over the phase of the signal.

   
    peak = abs(snr).numpy().argmax()
    snrpc= snr[peak]
    snrp = abs(snr[peak])
    time = snr.sample_times[peak]
   
    
    return snr,time,peak,snrpc,snrp

In [9]:
def gen_chisquare(temp,data,m_1,m_2,s_1,s_2,psd):
    
    n =int(0.72*pycbc.pnutils.get_freq('fSEOBNRv4Peak',m_1,m_2,s_1,s_2)**0.7)
    
    chisq = power_chisq(temp, data, n, psd=psd, low_frequency_cutoff=30.0)

    # convert to a reduced chisq
    chisq /= (n * 2) - 2

    return chisq

In [10]:
def snr_chisq_nsnr(num_templates,data_templates,psd,data,s):

    template_info = []

    for i in range(num_templates):
        T1 = data_templates[i]

        S1, time_bns, index_peak_bns, C_snrp, old_snr_bns = gen_SNR(T1,data,psd)
        S1.save(f"Signals_BBH_BNS/snr_complete_series/snr{i}.hdf")

        chisq_1 = gen_chisquare(T1, data, temp_bank[i][0], temp_bank[i][1], s, s, psd)
        chisq_val_bns = chisq_1[index_peak_bns]

        new_snr_bns = newsnr(abs(S1),chisq_1)

        snrp_bns = new_snr_bns[index_peak_bns]

        template_info.append([temp_bank[i][0],temp_bank[i][1],time_bns,old_snr_bns,snrp_bns,chisq_val_bns,C_snrp])

    return template_info

In [11]:
def recovery(data,n,data_templates,psd): 

    full__info =  snr_chisq_nsnr(n,data_templates,psd,data,0.8)

    info = {}
    for i in  full__info:
        info.update({i[4] : [i[0],i[1],i[2],i[3],i[5],i[6]]})

    snrp_1 = max(info.keys())
    time_1 = info.get(max(info.keys()))[2]
    m1_1 = info.get(max(info.keys()))[0]
    m2_1 = info.get(max(info.keys()))[1]
    chi_sq_1 = info.get(max(info.keys()))[4]
    Comp_snrp = info.get(max(info.keys()))[5]

    print("The signal detected is due to the following masses ",m1_1,"M and ", m2_1, "M at time ", time_1, "seconds", " with new SNR value as", snrp_1, " and chi~square value as ", chi_sq_1 )

    return m1_1,m2_1,time_1,snrp_1,full__info,Comp_snrp

In [12]:
def align_sub(m1_1,m2_1,time_1,com_snrp,data,psd):   
    
    # The time, amplitude, and phase of the SNR peak tell us how to align
    # our proposed signal with the data.

    # Shift the template to the peak time
    dt = time_1 - data.start_time 
    
    
    # if (m1_1 >= 15 and m2_1 >= 15) : 
    temp_sub  = gen_template('IMRPhenomPv2',m1_1,m2_1,0.8,0.8,data)

    # else :
    #     temp_sub = gen_template('TaylorF2',m1_1,m2_1,0.05,0.05,data)

    aligned = temp_sub.cyclic_time_shift(dt)

    # scale the template so that it would have SNR 1 in this data
    aligned /= sigma(aligned, psd=psd, low_frequency_cutoff=30.0)

    # Scale the template amplitude and phase to the peak value
    aligned = (aligned.to_frequencyseries() * com_snrp).to_timeseries()
    aligned.start_time = data.start_time

    subtracted = data - aligned

    psd2 = psd_data(subtracted)  

    
    return subtracted,psd2

In [13]:
data_templates = []

for i in range(20):
    T = timeseries.load_timeseries(f"Signals_BBH_BNS/signal_bbh{i}.hdf")
    data_templates.append(T)

for i in range(20):
    R = timeseries.load_timeseries(f"Signals_BBH_BNS/signal_bns{i}.hdf")
    data_templates.append(R)

temp_bank = pd.read_csv('Signals_BBH_BNS/Load_bbh_bns.csv')
temp_bank = temp_bank.values.tolist()

In [ ]:
for j in range(1):
    
    print("Recovery number:",j)
    print(' ')
    
    ts1 = timeseries.load_timeseries(f"Signals_BBH_BNS/Data_{j}.hdf")
    psd1 = psd_data(ts1)  
    mass_1,mass_2,time,snrp,total,j_snrp = recovery(ts1,40,data_templates,psd1)
    
    print(' ')


#     for i in range(5):

#         sub,psd_sub = align_sub(mass_1,mass_2,time,j_snrp,ts1,psd1)
#         M1,M2,T,SP,Tot,k_snrp = recovery(sub,200,data_templates,psd_sub)
#     #     pp.plot(sub.sample_times,sub)
#     #     pp.xlim(1.6,2.0)
#     #     pp.show()
#         mass_1=M1
#         mass_2=M2
#         time=T
#         snrp=SP
#         j_snrp = k_snrp
#         total.extend([[0,0,0,0,0,0]])
#         total.extend(Tot)

#         print(" ")
#     print("-----------------------------------------------------------------------------------")

In [14]:
T_B1 = data_templates[2]
T_B1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{2}.hdf")

In [ ]:
# df_total_info = pd.DataFrame(total,columns = ['M1','M2','Time','Old_SNR','New_SNR','Chi-Sq.','Comp_Snr'])
# df_total_info.to_excel('100_Inj_Rec.xlsx',index = False)

In [17]:
A = []
B = []
maxim_SNR = []

for i in range(40):

    print(i)

    if i != 2:

        cross = matched_filter(data_templates[i], T_B1,
                         psd=psd1, low_frequency_cutoff=30)

        T_N1_MF = timeseries.load_timeseries(f"Signals_BBH_BNS/snr_complete_series/snr{i}.hdf")

        A_B1 = (T_B1_MF - (cross*T_N1_MF))/(1-(cross**2))
        A.append(A_B1)

        A_N1 = (T_N1_MF - (cross*T_B1_MF))/(1-(cross**2))
        B.append(A_N1)

        norm = (A_B1**2) + (A_N1**2) + 2*(A_B1*A_N1*cross)
        SNR = ((A_B1*T_B1_MF) + ( A_N1 * T_N1_MF))/norm
        peak_SNR = abs(SNR).numpy().argmax()
        SNR_p = abs(SNR[peak_SNR])
        maxim_SNR.append([i,SNR_p])

In [18]:
M_SNR = 0
for i in range(len(maxim_SNR)):
    if M_SNR < maxim_SNR[i][1]:
        M_SNR = maxim_SNR[i][1]
        print(maxim_SNR[i][0])

0
3
4
11
33


In [19]:
print(maxim_SNR[32][1])

1.0000000000003952


In [20]:
print(maxim_SNR)

[[0, 1.000000000000099], [1, 1.0000000000000748], [3, 1.000000000000139], [4, 1.0000000000002458], [5, 1.0000000000000373], [6, 1.0000000000000921], [7, 1.0000000000000357], [8, 1.00000000000006], [9, 1.0000000000000577], [10, 1.0000000000000455], [11, 1.0000000000002645], [12, 1.00000000000016], [13, 1.0000000000001117], [14, 1.0000000000000586], [15, 1.0000000000000484], [16, 1.0000000000000642], [17, 1.0000000000000586], [18, 1.0000000000000449], [19, 1.0000000000000278], [20, 1.0000000000000153], [21, 1.0000000000000762], [22, 1.000000000000045], [23, 1.0000000000000615], [24, 1.0000000000000502], [25, 1.000000000000038], [26, 1.0000000000001483], [27, 1.0000000000000302], [28, 1.0000000000000564], [29, 1.0000000000000557], [30, 1.0000000000000118], [31, 1.000000000000026], [32, 1.0000000000000493], [33, 1.0000000000003952], [34, 1.0000000000000226], [35, 1.0000000000000195], [36, 1.0000000000000442], [37, 1.0000000000000413], [38, 1.0000000000000755], [39, 1.0000000000000837]]
